# Creador de archivos `.YAML` y `.job` para BOLTZ

## 1. Archivo `.fasta` a `.yaml`
El objetivo es crear archivos `.yaml` para meterselos como entrada a *BOLTZ* y generar modelos de las mutaciones. 

Los archivos fasta son la salida obtenida de AlphaFold Multimer. En estos archivos fasta "especiales", las secuencias de las cadenas están en la misma línea del archivo de texto, pero separadas por dos puntos `:`.

In [1]:
import pandas as pd
import os

In [32]:
def extract_sequences(fasta_file):
    complex_name = os.path.basename(fasta_file.split('.')[0])
    sequences = {complex_name: []}

    # Leer el archivo FASTA
    with open(fasta_file, 'r') as f:
        current_sequence = ''
        
        for line in f:
            line = line.strip()

            # Si la línea es un encabezado (empieza con '>'), ignorarla
            if line.startswith('>'):
                continue
            
            # Si la línea tiene contenido, añadirla a la secuencia actual
            if line:
                current_sequence += line

    if current_sequence:
        # Dividimos la cadena por ':' para obtener las secuencias individuales
        list_of_sequences = current_sequence.split(':')
        
        # Iteramos sobre la lista de secuencias y las añadimos a la lista del diccionario
        for seq in list_of_sequences:
            sequences[complex_name].append(seq)
            
    return sequences



In [34]:
print(extract_sequences('FASTA/1A22.fasta'))

{'1A22': ['PTIPLSRLFDNAMLRAHRLHQLAFDTYQEFEEAYIPKEQKYSFLQNPQTSLCFSESIPTPSNREETQQKSNLELLRISLLLIQSWLEPVQFLRSVFANSLVYGASDSNVYDLLKDLEEGIQTLMGRLEDGSPRTGQIFKQTYSKFDTNSHNDDALLKNYGLLYCFRKDMDKVETFLRIVQCRSVEGSCGF', 'SKEPKFTKCRSPERETFSCHWTDEVHHGTKNLGPIQLFYTRRNTQEWTQEWKECPDYVSAGENSCYFNSSFTSIWIPYCIKLTSNGGTVDEKCFSVDEIVQPDPPIALNWTLLNVSLTGIHADIQVRWEAPRNADIQKGWMVLEYELQYKEVNETKWKMMDPILTTSVPVYSLKVDKEYEVRVRSKQRNSGNYGEFSEVLYVTLPQMS']}


In [42]:
directorio = os.getcwd()+'/FASTA'
carpetas = os.listdir(directorio)
cplx_seqs = {}

for file in carpetas:
    fasta_path = os.path.join(directorio, file)
    fasta_name_and_seqs = extract_sequences(fasta_path)
    cplx_seqs.update(fasta_name_and_seqs)

In [54]:
list(cplx_seqs.keys())

['1LFD',
 '3M63',
 '3S9D',
 '2FTL',
 '5M2O',
 '3MZG',
 '1GUA',
 '1E50',
 '3EQS',
 '1K8R',
 '3EQY',
 '4BFI',
 '3Q8D',
 '1BP3',
 '1KNE',
 '5F4E',
 '4RS1',
 '1C1Y',
 '1A22',
 '3M62',
 '4OFY',
 '4G0N']

In [55]:
cplx_seqs['1LFD']

['PLYNQQVGDCCIIRVSLDVDNGNMYKSILVTSQDKAPTVIRKAMDKHNLDEDEPEDYELLQIISEDHKLKIPENANVFYAMNSAANYDFILKKRAFT',
 'MTEYKLVVVGAGGVGKSALTIQLIQNHFVDEYDPTIEDSYRKQVVIDGETCLLDILDTAGQEEYSAMRDQYMRTGEGFLCVFAINNTKSFEDIHQYREQIKRVKDSDDVPMVLVGNKCDLAARTVESRQAQDLARSYGIPYIETSAKTRQGVEDAFYTLVREIRQHKLRKLNPPD']

In [56]:
if not os.path.exists('YAML'):
    os.makedirs('YAML')
    
for cplx_name in list(cplx_seqs.keys()):
    with open(f'./YAML/{cplx_name}.yaml', 'w') as salida:
        seqs = cplx_seqs[cplx_name]
        salida.write(
f"""version: 1
sequences:
    - protein:
        id: A
        sequence: {seqs[0]}
    - protein:
        id: B
        sequence: {seqs[1]}"""
)

## Creación de `job.sh`

In [70]:
if not os.path.exists('Models'):
    os.makedirs('Models')

In [71]:
texto = """#!/bin/bash
#SBATCH -c 32
#SBATCH --mem-per-cpu=3952M
#SBATCH --gres=gpu:a100:1
#SBATCH -t 06:00:00
#SBATCH --job-name=Boltz_prueba2      # Nombre del trabajo (aparecerá en squeue)
#SBATCH --mail-type=BEGIN,END,FAIL  # Recibe correos al inicio, final y si falla
#SBATCH --mail-user=victor.monteagudo@icvv.es # ¡REEMPLAZA ESTO CON TU CORREO!

module --force purge
module load cesga/system boltz/2.1.1

"""


In [72]:
dir_YAML = os.listdir('YAML/')

for archivo_yaml in dir_YAML:
    if archivo_yaml.endswith('.yaml'):
        texto = texto + f'boltz predict "/home/csic/sbf/vmh/Boltz2/Luis/22dataset_SIN_muts_YAML/YAML/{archivo_yaml}" ' 
        texto = texto + '--out_dir /home/csic/sbf/vmh/Boltz2/Luis/22dataset_SIN_muts_YAML/Models --output_format pdb '
        texto = texto + '--use_msa_server --write_full_pae --write_full_pde '
        texto = texto + '--use_potentials --recycling_steps 10 --diffusion_samples 50'
        # texto = texto + '--step_scale 1.638 --num_workers 2 --subsample_msa'
        texto = texto + '\n'


In [73]:
with open('job.sh', 'w') as f:
    f.write(texto)

In [74]:
# prueba_job.sh

with open('prueba_job.sh', 'w') as f:
    f.write(
'''#!/bin/bash
#SBATCH -c 32
#SBATCH --mem-per-cpu=3952M
#SBATCH --gres=gpu:a100:1
#SBATCH -t 06:00:00
#SBATCH --job-name=Boltz_prueba2      # Nombre del trabajo (aparecerá en squeue)
#SBATCH --mail-type=BEGIN,END,FAIL  # Recibe correos al inicio, final y si falla
#SBATCH --mail-user=victor.monteagudo@icvv.es # ¡REEMPLAZA ESTO CON TU CORREO!

module --force purge
module load cesga/system boltz/2.1.1

boltz predict "/home/csic/sbf/vmh/Boltz2/Luis/22dataset_SIN_muts_YAML/YAML/1A22.yaml" --out_dir /home/csic/sbf/vmh/Boltz2/Luis/22dataset_SIN_muts_YAML/Models --output_format pdb --use_msa_server --write_full_pae --write_full_pde --use_potentials --recycling_steps 10 --diffusion_samples 50
'''
)